# League of Legends Project

## Part 1: Data Collection
    Ranked match data will be collected from each Summoner Name provided.
    Part 1A: Data Collection from Data Dragon
        - create mapping dictionaries from the json files provided
    Part 1B: Data Collection from Riot API
        - extract data from RIOT API
    
## Part 2: Data Analysis
    Exploratory Data Analysis will be done on the match data to discover basic statistics and trends.
    Part 2A: Data Wrangling and Cleaning
        - wrangle and manipulate data table for consistency
        - prepare data for visualization
    Part 2B: Data Visualization
        - visualize trends and correlations show in the data

# Part 1A

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import regex as re
import json
import pprint
import time
from bs4 import BeautifulSoup

## Champion JSON

In [2]:
def create_champion_mapping_dict(file_name):
    champion_key_dict = {}
    with open(f'json_files/{file_name}', encoding = 'utf-8') as f:
        champion_json = json.load(f)
        for champion, info in champion_json['data'].items():
            champion_name = champion
            for c_key, c_value in info.items():
                if c_key == 'key':
                    champion_key_dict[champion_name] = int(c_value)
    return champion_key_dict

champion_key_dict = create_champion_mapping_dict('champion.json')

## Item JSON

In [3]:
def create_item_mapping_dict(file_name):
    item_key_dict = {}
    with open(f'json_files/{file_name}', encoding = 'utf-8') as f:
        item_data = json.load(f)
        for item, info in item_data['data'].items():
            item_id = item
            for i_key, i_value in info.items():
                if i_key == 'name':
                    item_key_dict[int(item_id)] = i_value
    return item_key_dict

item_key_dict = create_item_mapping_dict('item.json')

## Queue JSON

In [4]:
def create_queue_mapping_dict(file_name):
    queue_info_dict = {}
    with open(f'json_files/{file_name}', encoding = 'utf-8') as f:
        queue_data = json.load(f)
        for queue in queue_data:
            queue_key = queue['queueId']
            queue_name = queue['description']
            queue_notes = str(queue['notes'])
            if 'deprecated' not in queue_notes.lower():
                queue_info_dict[queue_key] = queue_name
    return queue_info_dict

queue_info_dict = create_queue_mapping_dict('queue.json')

## Summoner Spell JSON

In [5]:
def create_summoner_spell_mapping_dict(file_name):
    ss_info_dict = {}
    with open(f'json_files/{file_name}', encoding = 'utf-8') as f:
        ss_data = json.load(f)
        for s_spell, info in ss_data['data'].items():
            for key in info:
                ss_key = info['key']
                ss_name = info['name']
                ss_description = info['description']
                ss_info_dict[ss_name] = (ss_key, ss_description)
    return ss_info_dict

ss_info_dict = create_summoner_spell_mapping_dict('summoner.json')

## Season JSON

In [6]:
def create_season_mapping_dict(file_name):
    season_info_dict = {}
    with open(f"json_files/{file_name}", encoding = "utf-8") as f:
        season_data = json.load(f)
        for season in season_data:
            season_info_dict[season['id']] = season['season']
    return season_info_dict
    
season_info_dict = create_season_mapping_dict('season.json')

# Part 1B

In [7]:
REQUEST_HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Charset': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'https://developer.riotgames.com',
    'X-Riot-Token': ''
    }

RIOT_API = ''

## Getting Account Information Using Summoner Name

In [8]:
def get_puu_id(summoner_name, riot_api_key, request_headers, access = True):
    summoner_name_updated = summoner_name.replace(' ', '%20')
    url = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'
    account_url = url + summoner_name_updated
    riot_headers = request_headers
    riot_headers["X-Riot-Token"] = riot_api_key
    while access:
        req = requests.get(account_url, headers = riot_headers)
        if req.status_code == 200:
            account_info = json.loads(req.content.decode("utf-8"))
            access = False
        else:
            time.sleep(10)

    return account_info['puuid']

## Getting a List of Game IDs (up to recent 100  solo rank games) using PUU ID

In [9]:
def get_match_ids(puu_id, riot_api_key, request_headers, start, end, access = True):
    match_ids = []
    match_history_url = f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puu_id}/ids?start={start}&count={end}'
    riot_headers = request_headers
    riot_headers["X-Riot-Token"] = riot_api_key

    while access:
        req = requests.get(match_history_url, headers = riot_headers)
        if req.status_code == 200:
            match_id_info = json.loads(req.content.decode("utf-8"))
            for match_id in match_id_info:
                match_ids.append(match_id)
            access = False
        else:
            time.sleep(10)

    return match_ids

## Get Match Information Using Match ID

In [10]:
def get_match_details(match_id, riot_api_key, request_headers, access = True):
    match_url = 'https://americas.api.riotgames.com/lol/match/v5/matches/' + str(match_id)
    riot_headers = request_headers
    riot_headers["X-Riot-Token"] = riot_api_key

    while access:
        req = requests.get(match_url, headers = riot_headers)
        if req.status_code == 200:

            match_details = json.loads(req.content.decode("utf-8"))
            access = False
        elif req.status_code == 404:
            print(f'{match_id} - DATA NOT FOUND - MATCH FILE NOT FOUND')
            match_details = '404'
            access = False
        else:
            print(req.status_code)
            print('Waiting for 2 minutes to refresh restriction request...')
            time.sleep(120)

    return match_details

## Cleaning and Reducing Match Data

In [11]:
def clean_and_reduce_match_data(match_details):
    match_cleaned_dict = {}
    for key, value in match_details['info'].items():

        if key == 'gameId':
            match_id = value

        if key == 'gameMode':
            match_type = value
            
        if key == 'participants':
            participant_details = value
        
        if key == 'queueId':
            queue_id = value
            match_cleaned_dict[(match_type, queue_id, match_id)] = participant_details
    
    return match_cleaned_dict

In [12]:
def filter_match_data_by_puuid(match_cleaned_dict, puu_id):
    for match_id, match_details in match_cleaned_dict.items():
        for participant_dict in match_details:
            if participant_dict['puuid'] == puu_id:
                return {match_id[0] + '_' + str(match_id[1]) + '_' + str(match_id[2]): participant_dict}

In [13]:
def remove_perks_attribute(match):
    for key, info in match.items():
        for key in list(info.keys()):
            if key == 'perks':
                del info[key]

    return match

## Turn Match Data into DataFrame

In [14]:
def make_match_df(match):
    match_df = pd.DataFrame.from_dict(match, orient = 'index')
    return match_df

## Create Super Function

In [15]:
def get_match_history_by_summoner_name(summoner_name, start, end, riot_api = RIOT_API, request_headers = REQUEST_HEADERS):
    puu_id = get_puu_id(summoner_name, riot_api, request_headers)
    match_ids = get_match_ids(puu_id, riot_api, request_headers, start, end)
    matches_df_listed = []

    for match_id in match_ids:
        match_detail = get_match_details(match_id, riot_api, request_headers)
        if match_detail != '404':
            match_cleaned = clean_and_reduce_match_data(match_detail)
            match_cleaned_filtered = filter_match_data_by_puuid(match_cleaned, puu_id)

            match_final = remove_perks_attribute(match_cleaned_filtered)

            match_df = make_match_df(match_final)
    
            matches_df_listed.append(match_df)
    
    return pd.concat(matches_df_listed)

## Testing with My Summoner Name

In [16]:
test = get_match_history_by_summoner_name('Nose Toucher', start = 0, end = 100)

NA1_3907855314 - DATA NOT FOUND - MATCH FILE NOT FOUND
NA1_3907855242 - DATA NOT FOUND - MATCH FILE NOT FOUND
NA1_3905639770 - DATA NOT FOUND - MATCH FILE NOT FOUND
NA1_3903729722 - DATA NOT FOUND - MATCH FILE NOT FOUND
NA1_3891842647 - DATA NOT FOUND - MATCH FILE NOT FOUND
429
Waiting for 2 minutes to refresh restriction request...


In [17]:
test.shape

(95, 101)

In [18]:
test

,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,doubleKills,dragonKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,goldSpent,individualPosition,inhibitorKills,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,neutralMinionsKilled,nexusKills,nexusLost,objectivesStolen,objectivesStolenAssists,participantId,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
ARAM_450_3932617357,17,0,0,32952,18,145,Kaisa,0,5,512,512,512,7237,10,0,2,0,False,False,False,False,False,False,18440,17400,Invalid,1,3,6655,3157,3020,3115,3003,3089,2052,19,4,13,MIDDLE,0,5,2,251,69675,33831,21855,0,0,1,0,0,6,0,43910,3411,1783,3507,86ATpC37LDRSm6xJmThE7BO7U1XrNIsYjD0kyC_Cq-1gmB...,0,,,SUPPORT,0,67,121,89,6,5,4,7,14,HdYGAKtKch1CyonnM0uEgJDP-B5iIcCo2D2FW50FlFbM24U,161,Nose Toucher,False,200,,0,1625,116923,38616,0,25059,3675,0,49,0,275,1,0,3337,1372,1420,0,4,0,0,0,0,0,False
ARAM_450_3932595197,18,0,0,16366,16,202,Jhin,0,2,73,73,73,5502,4,0,2,0,True,False,False,False,False,False,12588,11750,Invalid,0,2,3009,6671,3094,6676,0,1018,2052,13,2,11,NONE,796,5,3,487,14213,4024,4093,0,0,1,0,0,1,0,64522,15356,9234,3507,86ATpC37LDRSm6xJmThE7BO7U1XrNIsYjD0kyC_Cq-1gmB...,0,,,DUO,0,46,42,28,30,2,4,6,7,HdYGAKtKch1CyonnM0uEgJDP-B5iIcCo2D2FW50FlFbM24U,161,Nose Toucher,False,100,,35,1023,79432,20078,0,13465,3342,577,75,178,122,4,1,696,696,138,0,4,0,0,0,0,0,False
ARAM_450_3929271447,27,0,0,37675,18,21,MissFortune,0,5,198,198,198,12708,9,0,0,0,False,False,False,False,False,False,20462,18650,Invalid,0,2,6653,3157,4637,3135,3020,3089,2052,23,5,13,TOP,284,3,1,455,129037,48261,21068,0,0,0,0,0,1,0,38261,12697,7811,3507,86ATpC37LDRSm6xJmThE7BO7U1XrNIsYjD0kyC_Cq-1gmB...,0,,,SUPPORT,0,54,106,140,17,5,4,8,14,HdYGAKtKch1CyonnM0uEgJDP-B5iIcCo2D2FW50FlFbM24U,161,Nose Toucher,False,100,,41,1905,172119,65780,0,33433,3109,0,73,584,330,1,0,4822,4822,4553,0,4,0,0,0,0,0,True
ARAM_450_3929226937,27,0,1,21925,18,145,Kaisa,0,2,815,815,815,8602,5,0,1,0,True,False,True,False,False,False,15498,14100,Invalid,2,0,6655,3157,3115,3020,3089,1042,2052,16,3,16,MIDDLE,0,7,2,305,46364,31429,8304,0,0,0,0,0,6,0,26766,4719,5599,3507,86ATpC37LDRSm6xJmThE7BO7U1XrNIsYjD0kyC_Cq-1gmB...,0,,,SUPPORT,0,66,93,61,6,3,4,7,14,HdYGAKtKch1CyonnM0uEgJDP-B5iIcCo2D2FW50FlFbM24U,161,Nose Toucher,False,200,,0,1264,73596,36615,0,14036,3316,0,51,0,138,1,0,465,465,132,0,0,0,0,0,0,0,True
NEXUSBLITZ_1300_3929194179,13,0,0,15594,16,81,Ezreal,0,2,627,6319,627,5552,3,0,0,0,False,False,False,False,False,False,12664,12050,Invalid,0,0,3042,6691,3158,1036,3074,3133,3340,16,3,9,NONE,0,5,1,566,20799,9916,2715,4,0,0,0,0,4,0,43157,9343,6444,3507,86ATpC37LDRSm6xJmThE7BO7U1XrNIsYjD0kyC_Cq-1gmB...,0,,,SUPPORT,0,192,47,27,7,1,4,2,14,HdYGAKtKch1CyonnM0uEgJDP-B5iIcCo2D2FW50FlFbM24U,161,Nose Toucher,False,100,,5,1013,73130,19902,0,11558,3835,0,40,47,46,1,0,9173,642,2397,0,0,0,5,0,0,4,True
NEXUSBLITZ_1300_3929159505,17,0,0,20728,18,8